In [1]:
pip install pandas transformers sentence-transformers torch scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

# Load dataset
dff = pd.read_csv("final_data.csv")

# Select first 5000 rows
first_5k = dff.iloc[:5000]

# Select last 15000 rows
last_15k = dff.iloc[5000:]

# Take 500 random rows from the first 5k
subset_1 = first_5k.sample(n=500, random_state=42)

# Take 1500 random rows from the last 15k
subset_2 = last_15k.sample(n=1500, random_state=42)

# Combine both subsets
df = pd.concat([subset_1, subset_2])

# Shuffle final dataset (optional)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Print dataset size
print(df.shape)  # Should be (2000, columns)


(2000, 2)


In [9]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
# Load your dataset
#df = pd.read_csv("your_dataset.csv")  # Update with your file path
text_column = "text"  # Update with the actual column name in your dataset


In [13]:
# Load LaBSE model
model = SentenceTransformer("sentence-transformers/LaBSE")

In [15]:
# Ensure all values are strings and handle missing values
df[text_column] = df[text_column].fillna("").astype(str)

In [17]:
# Generate embeddings safely
df["embeddings"] = df[text_column].apply(lambda text: model.encode(text) if isinstance(text, str) and text.strip() else None)

In [19]:
# Example: Compute similarity between two news articles
similarity = cosine_similarity([df["embeddings"][0]], [df["embeddings"][1]])
print(f"Similarity Score: {similarity[0][0]}")

Similarity Score: 0.1864808350801468


In [29]:
df = df.dropna(subset=["embeddings", "label"])  # Remove rows with missing values

In [31]:
X = np.vstack(df["embeddings"].values)  # Convert embeddings into a NumPy array
y = df["label"].values  

print(f"X shape: {X.shape}, y shape: {y.shape}")  # Check sizes

X shape: (1994, 768), y shape: (1994,)


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
print(f"X shape: {X.shape}, y shape: {y.shape}")

X shape: (1994, 768), y shape: (1994,)


In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the classifier
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Check accuracy
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Accuracy: 0.8872


In [39]:
def predict_fake_news(text, model, clf):
    embedding = model.encode([text])  # Convert input text to an embedding
    prediction = clf.predict(embedding)[0]  # Predict using trained classifier
    return "Fake News" if prediction == 1 else "Real News"

In [41]:
new_text = "Government announces new tax policies for next year."
result = predict_fake_news(new_text, model, clf)
print(f"Prediction: {result}")

Prediction: Fake News


In [43]:
def predict_fake_news(text, model, clf):
    embedding = model.encode([text])  # Convert input text to an embedding
    prediction = clf.predict(embedding)[0]  # Predict using trained classifier
    return "Fake News" if prediction == 1 else "Real News"

# Ask for user input
while True:
    new_text = input("Enter news text (or type 'exit' to quit): ")
    if new_text.lower() == "exit":
        break
    result = predict_fake_news(new_text, model, clf)
    print(f"Prediction: {result}")

Enter news text (or type 'exit' to quit):  	text 3	a leaked video shows a group of probation officers brutally beating a teen inside a california detention center.the video, which was leaked to witness la by someone from inside the probation department, shows four probation officers beating the teen, while a supervisor looks on.according to witness la, the video was recorded inside barry j. nidorf juvenile hall, located in sylmar, california.the video has no sound, but it does show the 17-year-old talking to the supervisor moments before the attack. the footage does not show the teen threatening the officer in any way.a second officer rushes in from the hallway and begins hitting the teen from behind. as more officers rush into the room, the teenager is tackled and thrown to onto the cell s cot.the officers pile on top of him. video shows them punching the kid over and over again. at this point he is on his stomach, but his head and upper body are no longer on the cot.the supervisor re

Prediction: Fake News


Enter news text (or type 'exit' to quit):  	text 4	(reuters) - five democratic u.s. senators, including presidential candidate bernie sanders, told the justice department and federal communications commission on thursday that they had “significant concerns” about charter communications inc’s (chtr.o) planned acquisition of time warner cable inc twc.n and bright house networks. the senators, who also included elizabeth warren, ed markey, ron wyden and al franken, said in a letter to the agencies that are reviewing the proposal that the deal would create “a nationwide broadband duopoly,” with charter and comcast corp (cmcsa.o) in nearly two-thirds of u.s. high-speed broadband homes. the senators urged the government to thoroughly address all potential harm to consumers. they also raised concerns that charter will take on significant debt that could harm its ability to honor commitments to build out its network, including in rural areas. the fcc has said u.s. broadband prices are among th

Prediction: Real News


Enter news text (or type 'exit' to quit):  	text 7183	वाशिंगटन (रायटर) - दो यू.एस.सीनेटरों ने गुरुवार को संघीय अधिकारियों से आग्रह किया कि वे 1 बिलियन डॉलर के हवाई अड्डे के चेहरे की स्कैनिंग कार्यक्रम के नियोजित विस्तार को रोकें, उन्होंने कहा कि तकनीक नौ यू.एस. से प्रस्थान करने वाली कुछ उड़ानों पर यात्रियों की पहचान करने के लिए इस्तेमाल की गई थी।अंतर्राष्ट्रीय गंतव्यों के लिए हवाई अड्डे पर्याप्त सटीक नहीं हो सकते हैं और गोपनीयता की चिंताओं को बढ़ाते हैं।कांग्रेस ने गैर-अमेरिकी के लिए कार्यक्रम के उपयोग को मंजूरी दे दी है।नागरिकों, लेकिन कभी भी स्पष्ट रूप से अमेरिकियों के लिए इसके उपयोग को अधिकृत नहीं किया।होमलैंड सिक्योरिटी विभाग ने कहा है कि सिस्टम को यात्रियों को किसी और की पहचान का उपयोग करके देश छोड़ने से रोकने और संयुक्त राज्य अमेरिका में आगंतुकों को अपने वीजा को खत्म करने से रोकने के लिए आवश्यक है।सीनेटर माइक ली, एक रिपब्लिकन, और एडवर्ड मार्की, एक डेमोक्रेट, डिपार्टमेंट ऑफ होमलैंड के सुरक्षा सचिव किर्स्टजेन नीलसन को एक पत्र में, ने चिंता जताई कि बहुत से यात्रियों को दोषपूर्ण स्कै

...

Enter news text (or type 'exit' to quit):  	text 7187	సాయంత్రం 6 గంటలకు ప్రారంభమయ్యే బాయిలర్ గది యొక్క మరొక ప్రత్యక్ష ప్రసారం కోసం ప్రత్యామ్నాయ ప్రస్తుత రేడియో నెట్‌వర్క్ (ACR) కు ట్యూన్ చేయండిప్రతి బుధవారం 9 PM EST.బార్‌ఫ్లై తత్వవేత్తలు, తప్పుదారి పట్టించే నైతికవాదులు, మాసోకిస్టులు, వీధి మూలలో సువార్తికులు, మీడియా-మానియాక్, సావెంట్లు, రాజకీయ జంతువులు మరియు ప్రేమించదగిన రాస్కల్స్ కోసం సెన్సార్ చేయని, నిరంతరాయమైన టాక్ రేడియో కోసం మాతో చేరండి.21 వైర్, జేస్ విశ్లేషణ నుండి జే డయ్యర్, కాళి ట్రిబ్యూన్ ఫేమ్ యొక్క బ్రాంకో మాలిక్ మరియు ఈస్ట్ యొక్క ఆత్మ యొక్క డేనియల్ స్పాల్డింగ్.టునైట్ బాయిలర్ గ్యాంగ్ వేడుక మోడ్‌లో ఉంది, ఎందుకంటే మేము మా వన్ ఇయర్ ప్రసార వార్షికోత్సవాన్ని తాకినప్పుడు, ప్రశంసలు ఇవ్వబడతాయి, క్లాసిక్ క్లిప్‌లు ఆడబడతాయి మరియు బ్రస్సెల్స్, సిరియా, పామిరాలో కొత్త విశ్లేషణలు కనుగొనబడతాయి మేము జే డైర్ ఎ ఎ మీడియా ఫేకరీపైకి వెళ్తున్నాముసిబ్బంది, ఇది బ్రస్సెల్స్, పారిస్, బోస్టన్ లేదా శాన్ బెర్నార్డినోలో ఉన్నారు.మీరు పాల్గొనాలనుకుంటే, ప్రత్యామ్నాయ ప్రస్తుత రేడియో చాట్ గదిలో మాతో చేరండి బాయిల

Prediction: Fake News


Enter news text (or type 'exit' to quit):  	text 7191	வாஷிங்டன்.குடியரசுக் கட்சியின் ஜனாதிபதி காப்பீட்டாளர்களிடம், ஒபாமா கேர் என அழைக்கப்படும் சட்டத்திலிருந்து அமெரிக்கர்களை காப்பாற்றவும், சுகாதார விலைகளை குறைக்க முயற்சிக்கவும் அவர்கள் அனைவரும் ஒன்றிணைந்து செயல்பட வேண்டும் என்று கூறினார்.சட்டத்தை ரத்து செய்வதற்கான சுகாதாரத் திட்டத்தில் ஜனநாயகக் கட்சியினருடன் இணைந்து பணியாற்றுவதாக நம்புவதாக அவர் கூறினார், இது மில்லியன் கணக்கான காப்பீடு செய்யப்படாத அமெரிக்கர்களுக்கு பாதுகாப்பு அளித்தது.


Prediction: Real News


Enter news text (or type 'exit' to quit):  exit
